# Исследование надежности заемщиков


Задача проекта: на основе статистики о платёжеспособности клиентов исследовать, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

### Изучим общую информацию о данных

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Так, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных встречаются артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом является отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются значения -1 и 20.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

Выведем на экран количество строк-дубликатов в данных и удалим их.

In [15]:
data.duplicated().sum()

54

In [16]:
data = data.drop_duplicates()

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру. 

In [17]:
data['education'] = data['education'].str.lower()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Используем функцию `categorize_purpose()` и метод `apply()`. Для этого изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [30]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные.

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
def que(category):
    data_temp = data.pivot_table(index=category, values='debt', aggfunc=['count', 'sum', 'mean'])
    data_temp.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
    return data_temp
display(que('children'))


,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14107,1063,0.075353
1,4809,444,0.092327
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4146,385,0.092861
женат / замужем,12266,927,0.075575


**Вывод:** 
Распределение заемщиков-должников показывает, что лишь 7,5% бездетных заемщиков становится должниками, тогда как у клиентов кредитных организаций, имеющих детей, этот процент варьируется от 8,2% до 9,76%. В то же время, количество заемщиков с 3-4 детьми составляет крайне незначительную часть от общего числа заемщиков (около 1,8%), посему для выводов больше подходят данные по заемщикам с 1-2 детьми. Среди таких клиентов банков доля должников составляет 9,2-9,5%, что почти на 2% выше процента бездетных должников.
**Таким образом**, можно утверждать, что зависимость между количеством детей и возвратом кредита в срок существует: заемщики с детьми чаще становятся должниками по кредитам. Тем не менее, эта зависимость не выглядит очень значительной (7,5% против 9,5%).

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
display(que('family_status'))

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4146,385,0.092861
женат / замужем,12266,927,0.075575


**Вывод:** 
Мы видим, что процент должников от общего процента заемщиков колеблется от 6,6% (вдовцы) до 9,8% (неженатые и незамужние). Можно отметить, что процент должников выше в категориях, не познавших официального оформления семейных отношений (неженатые - 9,8% и живущие в гражданском браке - 9,3%), тогда как даже у разведенных или потерявших супруга процент значительно ниже (7,1% - в разводе, 6,6% вдовцы). Самая многочисленная категория заемщиков - люди, живущие в браке, показывают тоже довольно низкий процент просрочек - 7,6%. **Таким образом**, можно утверждать, что зависимость между семейным положением и возвратом кредита в срок есть и банкам было бы надежнее отдавать предпочтение тем заявителям, кто состоит или состоял в семейных отношениях.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
income_pivot = data.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
income_pivot.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
display(income_pivot)

# display(que('total_income_category'))

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15938,1353,0.084891
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** Как мы помним, чем ближе к началу алфавита категория дохода, тем выше доход заемщика: наивысший уровень дохода - у категории А, низший уровень - у категории Е. Тем не менее, прямой корреляции процента задолженностей с ростом или снижением уровня дохода заемщиков не наблюдается: самый низкий процент просрочек - у категории D (6%), при этом она соседствует с самыми высокими процентами долгов в соседних категориях C и E (8,5% и 9,1%). **Таким образом**, можно утверждать, что зависимости между уровнем дохода и возвратом кредита в срок нет.

Как разные цели кредита влияют на его возврат в срок?

In [31]:
purpose_pivot = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
purpose_pivot.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
purpose_pivot

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с автомобилем,4281,400,0.093436
операции с недвижимостью,10754,780,0.072531
получение образования,3989,369,0.092504
проведение свадьбы,2324,183,0.078744


**Вывод:** Исходя из вычислений выше, видим, что среди заемщиков, бравших кредит на операции с автомобилем или получение образования, процент задолженностей или просрочек выше (9,3% в обоих случаях, если округлять). Самые надежные заемщики - клиенты-"ипотечники" (7,3%). Немногим хуже и заемщики, планировавшие дополнительные расходы на свадьбу - 7,9%.

*Каковы возможные причины появления пропусков в исходных данных?* Если рассматривать данный датасет, где пропуски встречаются в столбцах "трудовой стаж" и "ежемесячный доход", можно предположить, что либо заемщики не предоставили часть данных о себе, что маловероятно, учитывая, что они уже получили кредит от банка, либо автоматизированная система сбора информации дала сбой. Нельзя исключать и человеческий фактор при переносе или обработке данных.

*Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных?* В изучаемом датасете пропуски были в двух столбцах: трудовой стаж и ежемесячный доход. Поскольку в таких данных, как доход, некоторые значения могут сильно выделяться среди большинства, это бы исказило данные при подсчете среднего значения. Использование медианы позволяет этого избежать, так как медианное значение - это такое значение, слева и справа от которого (больше и меньше которого) равное количество других значений. Таким образом, использование медианных значений для заполнения пропусков позволяет получить более взвешенные данные, что меньше исказит датасет.

## Общий вывод

Поскольку главная задача проекта - построение скоринговой модели для оценки платежеспособности потенциальных клиентов, анализ представленных данных позволяет сделать следующий вывод. Для оценки платежеспособности потенциальных заемщиков целесообразно оценивать наличие у заявителя детей, его семейное положение и предполагаемую цель кредита. Уровень дохода надо рассматривать категориально, так как чёткой корреляции между уровнем дохода и количеством просрочек выявлено не было.

Идеальным клиентом, таким образом, будет бездетный человек, состоящий или ранее состоявший в браке, и берущий кредит на недвижимость или (новую?!) свадьбу. 
Наиболее рискованным клиентом представляется холостой или сожительствующий с партнером человек с 1-2 детьми, который хочет приобрести автомобиль или получить образование.